# GPU

In [1]:
!nvidia-smi

Tue Dec  5 17:59:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup - Import Libraries

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install python-Levenshtein

import os

#uplink and import anvil server
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_MZ3TDKTVVS2XCPTYHYVKJLSZ-GNCNXRRUCBKA2OAL")

  Installing build dependencies ... canceledERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


# Upload Dataset (csv file)
* We will utilize the load_dataset library as it allows to split the dataset for training

In [3]:
# Upload file from Computer
from google.colab import files


uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
# HuggingFace Load Dataset
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "Eng_to_STE - Sheet1.csv"}, split="train") # set to train
dataset # display

In [ ]:
# Import Pandas Library
import pandas as pd
import io
# Read csv file
train_df = pd.read_csv(io.StringIO(uploaded['Eng_to_STE - Sheet1.csv'].decode('utf-8')))
train_df # display

# Format Dataset

In [ ]:
#format the dataset into one column for the model to read
def format_input(example: dict) -> str:
  example['formatted_input'] = f'''### Human: Please rewrite the following sentence using Standard Technical English guidelines: {example["Input"]}
  ### Machine: {example["Output"]}'''
  return example

dataset = dataset.map(format_input)
dataset

# Fine Tuning
* Prepare the Model (Falcon-7b): BitsAndBytes configuration

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

# Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# LoRA Configuration
* Training method that accelerates training time for large model while reducing memory usage.
* PEFT (Parameter Efficient Fine-Tuning), the training technique

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

# Loading the Trainer

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=False,  #set Gradient Checkpointing to FALSE
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="formatted_input",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Upcasting
* stablizes layers for training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# Train the Model


In [ ]:
trainer.train()

# Percentage Change

In [4]:
#calculates the amount of change from the original sentence to the converted sentence
import Levenshtein

#calculate percentage
def calculate_percentage(input_sentence, final_output):
  distance = Levenshtein.distance(input_sentence, final_output)
  percentage = (distance / ((len(final_output) + len(input_sentence))/2)) * 100
  return percentage

# Conversion Function
* connects to Anvil function

In [ ]:
import string


#alphanumeric function - verifies that the input is alphanumeric
def is_alphanumeric(input_string):
  allowed_chars = set(string.ascii_letters + string.digits + ' .')
  for char in input_string:
    if char not in allowed_chars:
      return False
  return True

#convesion function - sends the input into the model to generate an output
@anvil.server.callable
def convert_sentence(input_sentence):

  if not is_alphanumeric(input_sentence):
    return "Invalid input. Please enter only alphanumeric characters."
  else:
    #prompt that sets up the input sentence in a way that the model can read it
    prompt = f'''### Human: Please rewrite the following sentence into one sentence using Standard Technical English guidelines: {input_sentence}
  ### Machine: '''
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    print(tokenizer.decode(input_ids[0]))

    with torch.no_grad():
      max_length = min(2 * len(input_sentence), 50)
      output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, top_k=50)

      decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

      #removes the prompt from the output
      index = decoded_output.find('### Machine: ')
      clean_output = decoded_output[index + len('### Machine: '):].strip()

      #removes all non-alphanumeric characters
      final_output = ''.join(char for char in clean_output if char.isalnum() or char.isspace() or char == '.')

      #calls for the function that calculates the percentage of change
      percentage = calculate_percentage(input_sentence, final_output)

    return f"{final_output}\n\nPercentage of Change: {percentage}%"

# Feedback File
* create and write feedback to a file
* print any received feedback

In [ ]:
#feedback function that links to anvil
@anvil.server.callable
def new_feedback(feedback_input):
  file_path = 'feedback.txt'
  with open(file_path, 'a') as f:
    f.write(feedback_input)
    f.write('\n')
  return 'Thank you for your feedback!'

#function that reads feedback
def get_feedback():
  file_path = 'feedback.txt'
  if os.path.exists(file_path):
    with open(file_path, 'r') as f:
      feedback = f.read()
    return feedback
  else:
    return 'No feedback available.'


#display any received feedback
feedback_data = get_feedback()
fb_data = feedback_data.split('\n')

for line in fb_data:
  print(line)

* Server wait forever

In [ ]:
anvil.server.wait_forever()